## Imports

In [1]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F

## Model

In [2]:
class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1)
        self.bn1   = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.bn2   = nn.BatchNorm2d(128)
        self.pool  = nn.AvgPool2d(kernel_size=2, stride=2, padding=0)

        self.fc1   = nn.Linear(128 * 11 * 25, 512)
        self.fc2   = nn.Linear(512, 128)
        self.fc3   = nn.Linear(128, 10)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = self.pool(F.leaky_relu(self.bn1(self.conv1(x))))
        x = self.pool(F.leaky_relu(self.bn2(self.conv2(x))))
        x = x.view(x.size(0), -1)               
        x = F.leaky_relu(self.fc1(x))
        x = self.dropout(x)
        x = F.leaky_relu(self.fc2(x))
        x = self.fc3(x)                      
        return x

In [3]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def __len__(self):
        return len(self.x)
    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

In [4]:
def load_array(path_main: str):
    if os.path.isfile(path_main):
        return np.load(path_main, allow_pickle=False)
    base, ext = os.path.splitext(path_main)
    candidates = [path_main, base + ".npy", base + ".np"]
    for p in candidates:
        if os.path.isfile(p):
            return np.load(p, allow_pickle=False)

In [5]:
X_np = load_array("X_array.npy")  
Y_np = load_array("Y_array.npy")

X = torch.from_numpy(X_np).float()
Y = torch.from_numpy(Y_np).float()

## Saved weights

In [6]:
class_names = ["Flowers","Chocolate/Cacao","Coffee","Nuts","Fruits",
               "Citrus","Berries","Wood","Tobacco/Smoke","Herbs and spices"]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ConvNet().to(device)

state = torch.load("cnn_weights.pt", map_location=device)
model.load_state_dict(state)
model.eval()

ConvNet(
  (conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (fc1): Linear(in_features=35200, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=10, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

In [7]:
dataset_all = CustomDataset(X, Y)
loader = torch.utils.data.DataLoader(dataset_all, batch_size=64, shuffle=False)

all_probs = []
all_labels = []
with torch.no_grad():
    for xb, yb in loader:
        if xb.dim() == 3:              # (B,H,W) -> (B,1,H,W)
            xb = xb.unsqueeze(1)
        xb = xb.to(device).float()
        yb = yb.to(device).float()

        logits = model(xb)             # (B, C)
        probs  = torch.sigmoid(logits).cpu().numpy()
        all_probs.append(probs)
        all_labels.append(yb.cpu().numpy())

all_probs = np.concatenate(all_probs, axis=0)   # (N, C)
all_labels = np.concatenate(all_labels, axis=0) # (N, C)

pred_bin = (all_probs > 0.5).astype(int)

## Choose a wine sample

In [28]:
idx = 90  # change index
probs_1 = all_probs[idx].ravel()
preds_1 = pred_bin[idx].ravel()
true_1  = all_labels[idx].ravel()

print(f"\Sample #{idx}")
for name, p, z, t in zip(class_names, probs_1, preds_1, true_1):
    print(f"{name:<18} p={p:.3f}  pred={int(z)}  true={int(t)}")

\Sample #90
Flowers            p=0.381  pred=0  true=1
Chocolate/Cacao    p=0.209  pred=0  true=0
Coffee             p=0.161  pred=0  true=0
Nuts               p=0.634  pred=1  true=1
Fruits             p=0.208  pred=0  true=0
Citrus             p=0.691  pred=1  true=1
Berries            p=0.075  pred=0  true=0
Wood               p=0.007  pred=0  true=0
Tobacco/Smoke      p=0.072  pred=0  true=0
Herbs and spices   p=0.424  pred=0  true=0


In [29]:
acc_sample = (preds_1 == true_1).mean()
n_correct  = int((preds_1 == true_1).sum())
n_classes  = len(true_1)
print(f"Sample accuracy: {acc_sample:.4f}  ({n_correct}/{n_classes} correct labels)")

Sample accuracy: 0.9000  (9/10 correct labels)


In [9]:
# df = pd.DataFrame(all_probs, columns=[f"p_{n}" for n in class_names])
# for i, n in enumerate(class_names):
#     df[f"pred_{n}"] = pred_bin[:, i]
#     df[f"true_{n}"] = all_labels[:, i].astype(int)
# df.to_csv("cnn_predictions.csv", index=False)
# print("[saved] cnn_predictions.csv")